In [1]:
import pandas as pd
import numpy as np
import copy
import math
df = pd.read_csv('google-cluster-data-1.csv',sep=' ')

In [2]:
print(str(len(df)) + " tasks")
np.save("taskID.npy", df["TaskID"])
np.save("taskCPU.npy", df["NrmlTaskCores"])
np.save("taskMEM.npy", df["NrmlTaskMem"])

3535029 tasks


In [3]:
exec_time_arr = []
subtasks = 0
quantum = 300
for task in df.itertuples():
    taskID = task.TaskID
    burstTime = ((taskID%2)+1)*quantum
    exec_time_arr.append(burstTime)
    subtasks += burstTime / quantum

exec_time_frame = pd.DataFrame({'ExecutionTime': exec_time_arr})
df = df.join(exec_time_frame)
# From https://stackoverflow.com/questions/20602947/append-column-to-pandas-dataframe

np.save("exe_time_1.npy", df["ExecutionTime"])

In [4]:
print(subtasks)

5298274.0


In [5]:
df.sort_values(by=["Time"])

# Sort by time just in case the times are not sorted
# From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

times = list(dict.fromkeys(df["Time"].values)) # Removes duplicates from list while maintaining order
# From https://datagy.io/python-remove-duplicates-from-list/

In [6]:
prices = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]

def RR(task_queue, vms):
    vms = np.array([[np.float64(7), np.float64(11)]])
    vms = np.repeat(vms, repeats=100, axis=0)
    vm_id = 0
    reject_queue = []
    for task in task_queue:
        cpu_task = task.NrmlTaskCores
        mem_task = task.NrmlTaskMem
        cpu_vm = vms[vm_id][0]
        mem_vm = vms[vm_id][1]
        if cpu_task <= cpu_vm and mem_task <= mem_vm:
            cpu_vm = cpu_vm - cpu_task
            vms[vm_id][0] = cpu_vm
            mem_vm = mem_vm - mem_task
            vms[vm_id][1] = mem_vm
        else:
            nxt_vm = (vm_id + 1) % 100
            found = False
            while nxt_vm != vm_id:
                cpu_nxt_vm = vms[nxt_vm][0]
                mem_nxt_vm = vms[nxt_vm][1]
                if cpu_task <= cpu_nxt_vm and mem_task <= mem_nxt_vm:
                    cpu_nxt_vm = cpu_nxt_vm - cpu_task
                    vms[nxt_vm][0] = cpu_nxt_vm
                    mem_nxt_vm = mem_nxt_vm - mem_task
                    vms[nxt_vm][1] = mem_nxt_vm
                    found = True
                    break
                nxt_vm = (nxt_vm + 1) % 100
            if not found:
                reject_queue.append(task.TaskID)
        vm_id = (vm_id + 1) % 100
    
    return reject_queue, vms

def energy_consumption(vms, c, threshold, a, b):
    Pwr_ttl_t = 0
    for i in range(len(vms)):
        used_cpu = vms[i][0]
        cpu_cap = df["NrmlTaskCores"][i]
        Ur_m_t = 0
        if cpu_cap > 0:
            Ur_m_t = used_cpu / cpu_cap
        Pwr_m_st_t = c if Ur_m_t > 0 else 0
        Pwr_m_dy_t = None
        if Ur_m_t < threshold:
            Pwr_m_dy_t = Ur_m_t * a
        else:
            Pwr_m_dy_t = threshold * a + ((Ur_m_t - threshold) ** 2) * b
        Pwr_m_t = Pwr_m_st_t + Pwr_m_dy_t
        Pwr_ttl_t += Pwr_m_t
    return Pwr_ttl_t

def cost(Pwr_ttl_t, t, offset):
    price_idx = (math.floor(t - offset))/3600
    price = prices[int(price_idx)]
    total_cost_t = price * Pwr_ttl_t
    return total_cost_t

curr_task_id = 0
subtasks = 0
excess_burst_queue = []
all_reject_queue = []
vms = []
Pwr_ttl_all_t = 0
total_cost_all_t = 0
for t in times:
    task_queue = copy.deepcopy(excess_burst_queue)
    excess_burst_queue = []
    for task in task_queue:
        burst_time = task.ExecutionTime
        if burst_time > quantum:
            copied_task = task.copy()
            copied_task["ExecutionTime"] = burst_time - quantum
            excess_burst_queue.append(copied_task)
            task["ExecutionTime"] = quantum
        subtasks += 1
    while curr_task_id < len(df) and df["Time"][curr_task_id] == t:
        task = df.loc[curr_task_id]
        burst_time = task.ExecutionTime
        if burst_time > quantum:
            copied_task = task.copy()
            # Make deep copy of task for use in the next task queue https://www.programiz.com/python-programming/shallow-deep-copy
            copied_task["ExecutionTime"] = burst_time - quantum
            excess_burst_queue.append(copied_task)
            task["ExecutionTime"] = quantum
        task_queue.append(task)
        # Accessing single row by index: https://thispointer.com/select-rows-columns-by-name-or-index-in-dataframe-using-loc-iloc-python-pandas/
        subtasks += 1
        curr_task_id += 1
    (reject_queue, vms) = RR(task_queue, vms)
    all_reject_queue += reject_queue
    Pwr_ttl_t = energy_consumption(vms, 5, 0.5, 100, 200)
    Pwr_ttl_all_t += Pwr_ttl_t
    total_cost_all_t += cost(Pwr_ttl_t, t, 90000)

print(subtasks)

<ipython-input-6-2582b9b00fed>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task["ExecutionTime"] = quantum


5295934


In [7]:
if len(all_reject_queue) > 0:
    np.save("taskReject_1.npy", all_reject_queue)
np.save("VMs_1.npy", vms)
print(str(len(all_reject_queue)) + " rejections")

0 rejections


In [8]:
print("Total power consumption: " + str(Pwr_ttl_all_t))
print("Total cost: " + str(total_cost_all_t))

Total power consumption: 136012545681.75362
Total cost: 78238209969.12415


In [9]:
import pandas as pd
import numpy as np
import copy
df2 = pd.read_csv('google-cluster-data-1.csv',sep=' ')

In [10]:
print(str(len(df2)) + " tasks")
np.save("taskID_2.npy", df2["TaskID"])
np.save("taskCPU_2.npy", df2["NrmlTaskCores"])
np.save("taskMEM_2.npy", df2["NrmlTaskMem"])

3535029 tasks


In [11]:
exec_time_arr = []
subtasks = 0
quantum = 300
for task in df2.itertuples():
    taskID = task.TaskID
    burstTime = ((taskID%10)+1)*quantum
    exec_time_arr.append(burstTime)
    subtasks += burstTime / quantum

exec_time_frame = pd.DataFrame({'ExecutionTime': exec_time_arr})
df2 = df2.join(exec_time_frame)
# From https://stackoverflow.com/questions/20602947/append-column-to-pandas-dataframe

np.save("exe_time_2.npy", df2["ExecutionTime"])

In [12]:
print(subtasks)

19472018.0


In [13]:
df2.sort_values(by=["Time"])

# Sort by time just in case the times are not sorted
# From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

times = list(dict.fromkeys(df2["Time"].values)) # Removes duplicates from list while maintaining order
# From https://datagy.io/python-remove-duplicates-from-list/

In [14]:
prices = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]

def RR(task_queue, vms):
    vms = np.array([[np.float64(25), np.float64(40)]])
    vms = np.repeat(vms, repeats=100, axis=0)
    vm_id = 0
    reject_queue = []
    for task in task_queue:
        cpu_task = task.NrmlTaskCores
        mem_task = task.NrmlTaskMem
        cpu_vm = vms[vm_id][0]
        mem_vm = vms[vm_id][1]
        if cpu_task <= cpu_vm and mem_task <= mem_vm:
            cpu_vm = cpu_vm - cpu_task
            vms[vm_id][0] = cpu_vm
            mem_vm = mem_vm - mem_task
            vms[vm_id][1] = mem_vm
        else:
            nxt_vm = (vm_id + 1) % 100
            found = False
            while nxt_vm != vm_id:
                cpu_nxt_vm = vms[nxt_vm][0]
                mem_nxt_vm = vms[nxt_vm][1]
                if cpu_task <= cpu_nxt_vm and mem_task <= mem_nxt_vm:
                    cpu_nxt_vm = cpu_nxt_vm - cpu_task
                    vms[nxt_vm][0] = cpu_nxt_vm
                    mem_nxt_vm = mem_nxt_vm - mem_task
                    vms[nxt_vm][1] = mem_nxt_vm
                    found = True
                    break
                nxt_vm = (nxt_vm + 1) % 100
            if not found:
                reject_queue.append(task.TaskID)
        vm_id = (vm_id + 1) % 100
    
    return reject_queue, vms

def energy_consumption(vms, c, threshold, a, b):
    Pwr_ttl_t = 0
    for i in range(len(vms)):
        used_cpu = vms[i][0]
        cpu_cap = df2["NrmlTaskCores"][i]
        Ur_m_t = 0
        if cpu_cap > 0:
            Ur_m_t = used_cpu / cpu_cap
        Pwr_m_st_t = c if Ur_m_t > 0 else 0
        Pwr_m_dy_t = None
        if Ur_m_t < threshold:
            Pwr_m_dy_t = Ur_m_t * a
        else:
            Pwr_m_dy_t = threshold * a + ((Ur_m_t - threshold) ** 2) * b
        Pwr_m_t = Pwr_m_st_t + Pwr_m_dy_t
        Pwr_ttl_t += Pwr_m_t
    return Pwr_ttl_t

def cost(Pwr_ttl_t, t, offset):
    price_idx = (math.floor(t - offset))/3600
    price = prices[int(price_idx)]
    total_cost_t = price * Pwr_ttl_t
    return total_cost_t

curr_task_id = 0
subtasks = 0
excess_burst_queue = []
all_reject_queue = []
vms = []
Pwr_ttl_all_t = 0
total_cost_all_t = 0
for t in times:
    task_queue = copy.deepcopy(excess_burst_queue)
    excess_burst_queue = []
    for task in task_queue:
        burst_time = task.ExecutionTime
        if burst_time > quantum:
            copied_task = task.copy()
            copied_task["ExecutionTime"] = burst_time - quantum
            excess_burst_queue.append(copied_task)
            task["ExecutionTime"] = quantum
        subtasks += 1
    while curr_task_id < len(df2) and df2["Time"][curr_task_id] == t:
        task = df2.loc[curr_task_id]
        burst_time = task.ExecutionTime
        if burst_time > quantum:
            copied_task = task.copy()
            # Make deep copy of task for use in the next task queue https://www.programiz.com/python-programming/shallow-deep-copy
            copied_task["ExecutionTime"] = burst_time - quantum
            excess_burst_queue.append(copied_task)
            task["ExecutionTime"] = quantum
        task_queue.append(task)
        # Accessing single row by index: https://thispointer.com/select-rows-columns-by-name-or-index-in-dataframe-using-loc-iloc-python-pandas/
        subtasks += 1
        curr_task_id += 1
    (reject_queue, vms) = RR(task_queue, vms)
    all_reject_queue += reject_queue
    Pwr_ttl_t = energy_consumption(vms, 5, 0.9, 100, 200)
    Pwr_ttl_all_t += Pwr_ttl_t
    total_cost_all_t += cost(Pwr_ttl_t, t, 90000)

print(subtasks)

<ipython-input-14-8f7b3b834389>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task["ExecutionTime"] = quantum


18883760


In [15]:
if len(all_reject_queue) > 0:
    np.save("taskReject_2.npy", all_reject_queue)
np.save("VMs_2.npy", vms)
print(str(len(all_reject_queue)) + " rejections")

0 rejections


In [16]:
print("Total power consumption: " + str(Pwr_ttl_all_t))
print("Total cost: " + str(total_cost_all_t))

Total power consumption: 2052874977445.3335
Total cost: 1091755603619.9219


For a setup of 7 CPU units and 11 Mem units, we know that there are no rejections because after each time quantum, the VMs are reinitialized (i.e. the VMs are freeing up their space to make room for the incoming tasks). Thus, at each time quantum, the VMs just need to be able to service the incoming tasks (plus the old tasks that still need servicing as their execution time exceeds the time quantum). Because each task uses a small amount of CPU and mem and the number of tasks that the VMs need to service are now drastically lower, we thus know that there will be no rejections.

My code loads in the necessary CSV file into a df variable, and the number of tasks within the variable is printed out. Then, all the task IDs, task CPUs, and task memories are saved into their own respective files. Afterwards, I calculated the number of expected subtasks with burst time = ((TaskID % 2) + 1) * 300. Then, I sorted the tasks by time. For each time t, I constructed a task queue, pushing the tasks that have execution times that exceed the time quantum to the next task queue. I ran Round Robin on each task queue, reconstructing a 100x2 matrix (each CPU = 7 units, each Mem = 11 units) each time it was ran. I then calculated the energy consumption for all t for c = 5, dynamic power consumption threshold = 50%, a = 100, and b = 200. I also calculated the total cost for all t using a specified price array. I finally saved the VM matrix found after the last RR execution into VMs_1.npy.

I then repeated the process for different parameters (Burst time = ((TaskID % 10) + 1) * 300, each CPU = 25 units, each Mem = 40 units Threshold = 90%). For a setup of 25 CPU units and 40 Mem units, there are no rejections for the same reasons listed above. 

NOTE: I was unable to fully run the second setup and get the necessary output files. Please run the second setup and get the output file to compare with the solution.